# 应用调用
本文主要介绍如何通过阿里云百炼创建应用，并通过API或SDK调用应用。

### 创建应用
请参考[阿里云百炼官方文档](https://help.aliyun.com/document_detail/2782159.html?spm=a2c4g.2782133.0.i0)在*阿里云百炼控制台的应用中心->我的应用*创建应用。

以下示例展示通过CURL命令来调用应用的脚本。
注意: 需要使用您的API-KEY替换示例中的YOUR_API_KEY，请参考文档[获取API-KEY](https://help.aliyun.com/document_detail/2712195.html)，并将APP_ID替换示例中的YOUR_APP_ID，代码才能正常运行，APP_ID可以在“我的应用”中查看对应的应用ID。

In [ ]:
%%bash
curl --location 'https://dashscope.aliyuncs.com/api/v1/apps/YOUR_APP_ID/completion' \
--header 'Authorization: Bearer YOUR_API_KEY' \
--header 'Content-Type: application/json' \
--data '{
    "input": {
        "prompt": "如何做土豆炖猪脚?"
    },
    "parameters":  {}
}' --verbose

### 通过HTTP调用应用(流式)
以下示例展示通过CURL命令来调用应用，并通过流式的方式返回内容。
注意: 需要使用您的API-KEY替换示例中的YOUR_API_KEY，并将APP-ID替换示例中的YOUR_APP_ID，代码才能正常运行。

In [ ]:
%%bash
curl --location 'https://dashscope.aliyuncs.com/api/v1/apps/YOUR_APP_ID/completion' \
--header 'Authorization: Bearer YOUR_API_KEY' \
--header 'Content-Type: application/json' \
--header 'X-DashScope-SSE: enable' \
--data '{
    "input": {
        "prompt": "如何做土豆炖猪脚?"
    },
    "parameters":  {
        "incremental_output": "true"
    }
}' --verbose

### 通过SDK调用应用
注意: 需要使用您的API-KEY替换示例中的YOUR_API_KEY，并将APP-ID替换示例中的YOUR_APP_ID，代码才能正常运行。

In [ ]:
# 安装dashscope SDK。
!pip3 install dashscope

In [ ]:
# 通过环境变量设置API-KEY
%env DASHSCOPE_API_KEY=YOUR_AP_KEY

# 通过代码参数设置API-KEY
# from dashscope import Application
# Application.call(api_key='YOUR_API_KEY')

In [ ]:
from http import HTTPStatus
from dashscope import Application


def app_call():
    response = Application.call(app_id='YOUR_APP_ID',
                                prompt='API接口说明中, TopP参数改如何传递?',
                                )

    if response.status_code != HTTPStatus.OK:
        print('request_id=%s, code=%s, message=%s\n' % (response.request_id, response.status_code, response.message))
    else:
        print('request_id=%s\n output=%s\n usage=%s\n' % (response.request_id, response.output, response.usage))


if __name__ == '__main__':
    app_call()

### 多轮对话
阿里云百炼应用提供云端托管多轮对话功能，通过session_id进行多轮会话，阿里云百炼云端将自动托管多轮会话，调用侧无需自行维护多轮会话。

下面的例子中，第一次调用后返回session_id，在第二次调用时，可以传入第一次返回的session_id，这样第二次调用模型服务时，将携带第一次调用的会话信息。

说明: 目前session id会话有效期是1个小时，最大历史会话轮数为50。

In [ ]:
from http import HTTPStatus
from dashscope import Application


def call_with_session():
    response = Application.call(app_id='YOUR_APP_ID',
                                prompt='我想去新疆',
                                )

    if response.status_code != HTTPStatus.OK:
        print('request_id=%s, code=%s, message=%s\n' % (response.request_id, response.status_code, response.message))
        return

    response = Application.call(app_id='YOUR_APP_ID',
                                prompt='那边有什么旅游景点或者美食?',
                                session_id=response.output.session_id
                                )
    if response.status_code != HTTPStatus.OK:
        print('request_id=%s, code=%s, message=%s\n' % (response.request_id, response.status_code, response.message))
    else:
        print('request_id=%s, output=%s, usage=%s\n' % (response.request_id, response.output, response.usage))


if __name__ == '__main__':
    call_with_session()

### 流式输出
流式输出需要添加对应参数`stream=True`。如果需要增量输出，需要设置`incremental_output=True`。



In [ ]:
from http import HTTPStatus
from dashscope import Application


def call_with_stream():
    responses = Application.call(app_id='YOUR_APP_ID',
                                 prompt='如何做炒西红柿鸡蛋？',
                                 stream=True,
                                 incremental_output=True
                                 )

    for response in responses:
        if response.status_code != HTTPStatus.OK:
            print('request_id=%s, code=%s, message=%s\n' % (
                response.request_id, response.status_code, response.message))
        else:
            print(response.output.text, end='')


if __name__ == '__main__':
    call_with_stream()